In [ ]:
from numpy.random import uniform, randint
from math import sqrt, pi
import cmath 
from collections import namedtuple
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

# Parameters

In [ ]:
N = 200 # approximate number of disks 
η =  0.7 # density
r = sqrt(η / (N * pi)) # radius 
ε = 0.2 * r # shift

# Initial configuration

In [ ]:
Point = namedtuple('Point', 'x y')

In [ ]:
def init_centers_honeycomb():
    centers = []
    k = int(sqrt(2.0 * N / sqrt(3.0)))
    l = int(k * sqrt(3.0) / 2.0 + 0.5)
    for i in range(k):
        for j in range(l):
            centers.append(Point((2.0 * i + 1.0)/(2.0 * k), (2.0 * j + 1.0 + (i % 2))/(2.0 * l)))    
    return centers

def init_centers_grid():
    centers = []
    N_sqrt = round(sqrt(N))
    for i in range(N_sqrt):
        for j in range(N_sqrt):
            centers.append(Point((2.0 * i) / (2.0 * N_sqrt), 2.0 * j / (2.0 * N_sqrt)))    
    return centers

def show_configuration(centers, r):
    fig = plt.figure(figsize=(6, 6))
    for i in range(len(centers)):
        center = centers[i]
        for ix in range(-1, 2):
            for iy in range(-1, 2):
                plt.gca().add_patch(plt.Circle((center.x + ix, center.y + iy), radius = r,  fc = 'r'))
    plt.axis('scaled')
    plt.axis([0.0, 1.0, 0.0, 1.0])

In [ ]:
show_configuration(init_centers_honeycomb(), r)

In [ ]:
show_configuration(init_centers_grid(), r)

# MCMC Sampling

In [ ]:
def dist(pt1, pt2): # on torus
    d_x = abs(pt1.x - pt2.x) % 1.0
    d_x = min(d_x, 1.0 - d_x)
    d_y = abs(pt1.y - pt2.y) % 1.0
    d_y = min(d_y, 1.0 - d_y)
    return  sqrt(d_x**2 + d_y**2)

In [ ]:
def move_center(cur_center, ε):
    return Point((cur_center.x + uniform(-ε, ε)) % 1.0, (cur_center.y + uniform(-ε, ε)) % 1.0)

def MCMC(n_steps):
    centers = init_centers_grid() 
    #centers = init_centers_honeycomb()
    for step in range(n_steps):
        rand_idx = randint(len(centers))
        cur_center = centers[rand_idx]
        new_center = move_center(cur_center, ε)
        
        min_dist = min(dist(new_center, c) for c in centers if c != cur_center)
        if (min_dist >= 2.0 * r):
            centers[rand_idx] = new_center
    return centers

In [ ]:
centers = MCMC(5000)
show_configuration(centers, r)